In [1]:
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.subplots as sp
import plotly.express as px
import numpy as np

from solution import *

In [2]:
n_colors = 10

colors = px.colors.sample_colorscale("turbo", [n/(n_colors -1) for n in range(n_colors)])

In [3]:
def format_face_sols(face:Face):
    mid_domain = 25
    face.get_face_sols()
    return np.array(face.sols), np.repeat(mid_domain, len(face.sols))

def create_node_data(face_list:[Face]):
    node_data = {}
    for face in face_list:
        x, y = format_face_sols(face)

        if face.axis == Axes.X:
            node_data[face.name] = [x, y]
        elif face.axis == Axes.Y:
            node_data[face.name] = [y, x]
        else: 
            pass
    return node_data


In [12]:
def show_solution_step(s:Solution):
    node_viz_data = []

    for ix, node in enumerate(s.graph.nodes):
            node_viz_data.append(create_node_data(s.graph.nodes[ix]["props"].faces.face_list))

    node_names = [str(ix) for ix in s.graph.nodes]
    
    fig = sp.make_subplots(rows=1, cols=len(node_viz_data), subplot_titles=node_names, shared_yaxes=True, shared_xaxes=True,)

    for ix, faces in enumerate(node_viz_data):
        showlegendbool = True if ix == 0 else False
        for iy, (face_name, face_coords) in enumerate(faces.items()):
            fig.add_trace(go.Scatter(x=face_coords[0], y=face_coords[1], name=face_name, marker_color=colors[iy], showlegend=showlegendbool), row=1, col=ix+1)


    fig.show()

In [13]:
s = Solution()
show_solution_step(s)

In [6]:
s.spanning_tree

[(0, 1), (1, 2), (2, 3), (3, 1), (3, 0), (2, 0)]

In [14]:
s.step_through_problem()
show_solution_step(s)

ic| solution.py:39 in solve_2D_problem_step()- ix: 0, edge: (0, 1)
ic| solution.py:46 in solve_2D_problem_step()
    f"{edge, orient}": '((0, 1), <Orient.NORTH: 1>)'


In [15]:
s.step_through_problem()
show_solution_step(s)

ic| solution.py:39 in solve_2D_problem_step()- ix: 1, edge: (1, 2)
ic| solution.py:46 in solve_2D_problem_step()
    f"{edge, orient}": '((1, 2), <Orient.SOUTH: 2>)'


In [16]:
s.step_through_problem()
show_solution_step(s)

ic| solution.py:39 in solve_2D_problem_step()- ix: 2, edge: (2, 3)
ic| solution.py:46 in solve_2D_problem_step()
    f"{edge, orient}": '((2, 3), <Orient.EAST: 3>)'


In [17]:
s.step_through_problem()
show_solution_step(s)

ic| solution.py:39 in solve_2D_problem_step()- ix: 3, edge: (3, 1)
ic| solution.py:46 in solve_2D_problem_step()
    f"{edge, orient}": '((3, 1), <Orient.WEST: 4>)'
/Users/julietnwagwuume-ezeoke/_UILCode/gqe-phd/xie2023/actions.py:75: UserWarning:

Secondary relate failed

ic| solution.py:57 in solve_2D_problem_step()
    "trying spatial relate again": 'trying spatial relate again'
ic| actions.py:47 in secondary_relate()
    "yyy": 'yyy'
    face_i1.sols: [14, 13]
    face_i2.sols: [15, 14]
    set1: [12, 11]
    set2: [14, 13]
    face_j.getSolutions(): [{<Axes.Y: 1>: 13}]


## testing 

In [9]:
node_viz_data = []

for ix, node in enumerate(s.graph.nodes):
        node_viz_data.append(create_node_data(s.graph.nodes[ix]["props"].faces.face_list))

# node_viz_data

In [10]:
node_names = [str(ix) for ix in s.graph.nodes]
fig = sp.make_subplots(rows=1, cols=len(node_viz_data), subplot_titles=node_names)

for ix, faces in enumerate(node_viz_data):
    showlegendbool = True if ix == 0 else False
    for iy, (face_name, face_coords) in enumerate(faces.items()):
        fig.add_trace(go.Scatter(x=face_coords[0], y=face_coords[1], name=face_name, marker_color=colors[iy], showlegend=showlegendbool), row=1, col=ix+1)


fig.show()